<a href="https://colab.research.google.com/github/RalphEng/Colab/blob/main/Z4Y_newWhisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Made by **Z4Y**

Google drive files transcribing with OPENAI Whisper

**Note: - ONLY WORKS WITH AUDIO THAT HAS MONO TYPE**

STEPS:

1/ Get Google Colab resources (https://colab.research.google.com/signup) [100 points is more than enough for 40+ transcriptions (needed for large-v2 modal)]

2/ Write file location path & File name (plus filetype) in variable 'path'.

3/ Set Number of speakers, Language & model

4/ Run code [under options 'Runtime', select 'Run all'.] [You can find 'Runtime' at the top of the screen, next to 'Insert' & 'Tools']

5/ Download .txt file from left menu option [the one with the folder icon] named; 'Files'.


How to videos:
  1. Getting started video [Not necessary to run code] : https://youtu.be/yVLhG4-7Sj4
  2. Audacity set MONO audio: https://www.youtube.com/watch?v=TTbBibBDGpg&ab_channel=LearnAudacity

*Note: This requires giving the application permission to connect to your drive. Only you will have access to the contents of your drive, but please read the warnings carefully.*

###**For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*

In [12]:


from google.colab import drive
drive.mount('/content/drive')
# Write file drive location & filename below!
path = '/content/drive/MyDrive/AI Test/Whisper/FreitagUm08-18.wav'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install git+https://github.com/openai/whisper.git
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-endmahwd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-endmahwd
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=e371fb72abee7b31af4d56947f5e19856a0f2bb02a2eab20c52ee2daac79f668
  Stored in directory: /tmp/pip-ephem-wheel-cache-x884en8a/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [3]:

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [14]:
# Set Number of speakers, Language & model

num_speakers = 2 #@param {type:"integer"}

language = "German" #@param ["any", "English", "Dutch", "German"]

model_size = "large-v2" #@param ["tiny", "base", "small", "medium", "large", "large-v2"]


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

In [15]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'
  model = whisper.load_model(model_size)

In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.87G/2.87G [00:33<00:00, 92.0MiB/s]


In [17]:
result = model.transcribe(path)
segments = result["segments"]

In [18]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [19]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [20]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [21]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [22]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w", encoding="utf-8")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()